In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

A UK based e-commerce company want to divide their customer into egments and develop marketing strategies according to this segmentation results. 
Data Set Story:

https://archive.ics.uci.edu/ml/datasets/Online+Retail+II 

This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.

Item listed are souvenirs and gift item.

Majority of the customers  are wholesale traders.

Attribute Information:

    • InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.\n
    • StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each product.\n
    • Description: Item name and description. Nominal.\n
    • Quantity: The quantities of each  item per transaction. Numeric.\n
    • InvoiceDate: Invoice date and time. Numeric.\n
    • UnitPrice: Unit price. Numeric. Product price per unit (£).\n
    • CustomerID: Customer number. Nominal. A 5-digit  number uniquely assigned to each customer.\n
    • Country: Country name. Nominal. The name of the country where a customer resides.

In order to import excel files xlrd and opepyxl shoud be installed. By using pip indtall you can isntall them 

In [ ]:
!pip install xlrd


In [ ]:
!pip install openpyxl

Firs step is importing needed libraries: numpy, pandas and datetime


In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

Then loading the dataset.

In [ ]:

df_ = pd.read_excel("../input/uci-online-retail-ii-data-set/online_retail_II.xlsx", sheet_name="Year 2010-2011" ) 
df = df_.copy()
df.head()

After loading data set, we are trying to get general information about our dataset.

In [ ]:
df.info()

Total Number of missing values

In [ ]:
df.isnull().sum()

Total number of unique items

In [ ]:
df["StockCode"].nunique()

Total quantity for each item

In [ ]:
df["StockCode"].value_counts().head()

Items ordered most

In [ ]:
df.groupby("StockCode").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head

Total number of invoices

In [ ]:
df["Invoice"].nunique()

# Data Preprocessing

After getting familiar with the dataset, we are preparing data for analysis. For this case we are going to drop null values, clear out thereturned items. For this analysis outliers are neglected so we are not going to do any operation about outliers. 

Dropping out null values

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

Invocies starting with "C" are either cancelled or returned goods so we are dropping out the item from the data set.

In [ ]:
df=df[~df["Invoice"].str.contains("C", na=False)]

We also clear any items Quantity less than 1

In [ ]:
df= df[(df["Quantity"]>0)]

In [ ]:
df.describe().T

# RFM

We are startig with addin TotalPrice column

In [ ]:
df["TotalPrice"]=df["Quantity"]*df["Price"]

In [ ]:
df.head()

## Recency


To calculate Recency value, we nned last transaction date. 
We fix the analysis date 2 days after the last transaction date to get resanable recency values. 

Last transaction date

In [ ]:
df["InvoiceDate"].max()

In [ ]:
today_date = dt.datetime(2011, 12, 11)

We are forming a Data Frame for Customer ID withthe variables of InvoiceDate(will be changed to Rececny in next step), Invoice number(will be changed to "Frequency") and Total sum of transactions(will be changed to "Monetary")

In [ ]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: num.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

We reassing the column names in previous cell. 
Recency: We subtracted today's date from the last transation date to calculate recency
Frequency: We counted invoiced per each customer. Since each invoice refers to one transaction we can use them as frequency.
Monetary: We sum up all transations per each customer as their monetary value.

In [ ]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']

To avoid miscalculation we run a acheck querry to eleminate 0 values in Monetary and Frequency

In [ ]:
rfm = rfm[(rfm["Monetary"]) > 0 & (rfm["Frequency"] > 0)]

In [ ]:
rfm

We divide all the score into 5 categories with qcut function where 5 is the best and 1 is the worst.
For Frequency most recent transaction is the best.

In [ ]:
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

In [ ]:
rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [ ]:
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

After calculating  RFM scores, we  combine scores as a new column

In [ ]:
rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) +
                    rfm['MonetaryScore'].astype(str))

In [ ]:
rfm

We create segment table.

In [ ]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

We add a new column to rfm dataframe as "Segment"

In [ ]:
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)

In [ ]:
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)

In [ ]:
rfm

General statistics for each segment

In [ ]:
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])

In [ ]:
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count",
                                                                             "min", "median", "max"])

We can expoert desired segments and customer number as csv wıth following codes

In [ ]:
#new_df = pd.DataFrame()

In [ ]:
#new_df["Loyal_Customers"] = rfm[rfm["Segment"] == "Loyal_Customers"].index

In [ ]:
#new_df["Loyal_Customers"] = rfm[rfm["Loyal_customers"].astype(int)

In [ ]:
#new_df.to_excel("Loyal_Customers.xlsx")